In [6]:
# PyTorch 예제

In [2]:
! nvidia-smi

Mon May  4 05:04:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 440.82       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 980 Ti  On   | 00000000:01:00.0 Off |                  N/A |
|  0%   57C    P8    31W / 300W |     11MiB /  6083MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
from torch import cuda
assert cuda.is_available()
assert cuda.device_count() > 0
print(cuda.get_device_name(cuda.current_device()))

GeForce GTX 980 Ti


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 10)

    def forward(self, x):
        x = x.float()
        h1 = F.relu(self.fc1(x.view(-1, 784)))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        h4 = F.relu(self.fc4(h3))
        h5 = F.relu(self.fc5(h4))
        h6 = self.fc6(h5)
        return F.log_softmax(h6, dim=1)

print("init model done")

batch_size = 64
test_batch_size = 1000
epochs = 5
lr = 0.01
momentum = 0.5
no_cuda = True
seed = 1
log_interval = 200

use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
print("set vars and device done")

transform = transforms.Compose([
                 transforms.ToTensor(),
                 transforms.Normalize((0.1307,), (0.3081,))])

train_loader = torch.utils.data.DataLoader(
  datasets.MNIST('./', train=True, download=True, 
                 transform=transform), 
    batch_size = batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./', train=False, download=True,
                 transform=transform), 
    batch_size=test_batch_size, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)


def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(log_interval, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() 
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format
          (test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    
for epoch in range(1, epochs):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(log_interval, model, device, test_loader)

    

init model done
set vars and device done
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313337
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.302358
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.262844
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.994884
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.259129

Test set: Average loss: 0.7034, Accuracy: 7826/10000 (78%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.661289
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.495438
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.431124
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.396521
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.284446

Test set: Average loss: 0.2711, Accuracy: 9257/10000 (93%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.199460
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.102232
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.227585
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.063849
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.118189

Test set: Average loss: 0.1998, Accuracy: 9433/10000 (94%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.254